In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

# Model

In [ ]:
input_length = 8
input_size = 36
input_class = 4
hidden_layer1 = 64
hidden_layer2 = 128

In [ ]:
class Model1() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer1)
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, cell2])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense = tf.layers.dense(inputs=output, units=input_class)
            self.logits = dense

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc
                 

In [ ]:
class Model2() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
  
    def build(self) :
        with tf.variable_scope(self.name) :
            
            self.X = tf.placeholder(tf.float32, [None, input_length, input_size])
            self.Y = tf.placeholder(tf.float32, [None, input_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            cell1 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            dropout1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
            cell2 = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer2)
            multi_cell = tf.nn.rnn_cell.MultiRNNCell([dropout1, cell2])
            
            output, state = tf.nn.dynamic_rnn(multi_cell, self.X, dtype=tf.float32)
            output = tf.transpose(output,[1,0,2])[-1]
            
            dense = tf.layers.dense(inputs=output, units=input_class)
            self.logits = dense

            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)

            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, X_input, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: X_input, self.training: training})

    def get_accuracy(self, X_input, Y_input, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: X_input,self.Y: Y_input, self.training: training})

    def train(self, X_input, Y_input, learning_rate,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate:learning_rate,self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
                
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc